In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tsmoothie.smoother import GaussianSmoother
import spikeinterface
import spikeinterface.full as si
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw
import spikeinterface.postprocessing as sp
import spikeinterface.preprocessing as spre
import spikeinterface.qualitymetrics as qm
import helper_functions as helper
from spikeinterface.sorters import run_sorter_local

In [2]:
#Reading the file, BP filtering
#path of the network file
local_path= '/mnt/disk20tb/PrimaryNeuronData/Maxtwo/FolicAcid/FolicAcid/231005/M05506/Network/000055/data.raw.h5' #network data from chip 16848

recording1 = se.read_maxwell(local_path,stream_id='well001') #well000 - well005 for maxtwo 



#recording = si.ConcatenateSegmentRecording([recording1,recording2])
channel_ids = recording1.get_channel_ids()
fs = recording1.get_sampling_frequency()
num_chan = recording1.get_num_channels()
num_seg = recording1.get_num_segments()
total_recording = recording1.get_total_duration()

#print('Channel ids:', channel_ids)
print('Sampling frequency:', fs)
print('Number of channels:', num_chan)
print('Number of segments:', num_seg)
print(f"total_recording: {total_recording} s")

recording_bp = spre.bandpass_filter(recording1, freq_min=300, freq_max=3000)

recodring_cmr = spre.common_reference(recording_bp, reference='global', operator='median')
#recording_chunk = recodring_cmr.frame_slice(start_frame= 1*fs,end_frame=425*fs)
recording_chunk = recodring_cmr.frame_slice(start_frame= 0*fs,end_frame=180*fs)
print(f"chunk duration: {recording_chunk.get_total_duration()} s")

Sampling frequency: 10000.0
Number of channels: 1019
Number of segments: 1
total_recording: 300.08 s
chunk duration: 180.0 s


## RUnnig sorting

In [4]:
default_KS2_params = ss.get_default_sorter_params('kilosort2')
print(default_KS2_params)
default_KS2_params['keep_good_only'] = True
# default_KS2_params['detect_threshold'] = 12
# default_KS2_params['projection_threshold']=[18, 10]
# default_KS2_params['preclust_threshold'] = 8
run_sorter = ss.run_kilosort2(recording_chunk, output_folder="./sorting/FolicAcid10mg", docker_image= "si-98-ks2-maxwell",verbose=True, **default_KS2_params)

## if running on NERSC:
#run_sorter_local("kilosort2",recording_chunk, output_folder="./sorting/FolicAcid10mg", delete_output_folder=False,verbose=True,with_output=True,**default_KS2_params)
#run_sorter = ss.run_sorter('kilosort2',recording= recording_chunk, output_folder="/mnt/disk15tb/mmpatil/Spikesorting/sorter_output/kilosort2",docker_image= True,verbose=True, **default_KS2_params)

{'detect_threshold': 6, 'projection_threshold': [10, 4], 'preclust_threshold': 8, 'car': True, 'minFR': 0.1, 'minfr_goodchannels': 0.1, 'freq_min': 150, 'sigmaMask': 30, 'nPCs': 3, 'ntbuff': 64, 'nfilt_factor': 4, 'NT': None, 'AUCsplit': 0.9, 'wave_length': 61, 'keep_good_only': False, 'skip_kilosort_preprocessing': False, 'scaleproc': None, 'save_rez_to_mat': False, 'delete_tmp_files': True, 'delete_recording_dat': False, 'n_jobs': 48, 'chunk_duration': '1s', 'progress_bar': True}


/tmp/ipykernel_1017136/2961592890.py:7: DeprecationWarning: run_kilosort2 is deprecated. Use run_sorter(sorter_name='kilosort2') instead.
  run_sorter = ss.run_kilosort2(recording_chunk, output_folder="./sorting/FolicAcid10mg", docker_image= "si-98-ks2-maxwell",verbose=True, **default_KS2_params)


Starting container
spikeinterface is already installed in si-98-ks2-maxwell
Installing extra requirements: ['neo', 'h5py']
Running kilosort2 sorter inside si-98-ks2-maxwell
Stopping container


In [3]:

# loading the KS2 sorted object
sorting_KS3 = ss.Kilosort2Sorter._get_result_from_folder('./sorting/FolicAcid10mg/sorter_output/')
total_units = sorting_KS3.get_unit_ids()
print(len(total_units))
#print(len(total_units))
channel_ids = recording_chunk.get_channel_ids()
sorting_good = sorting_KS3.remove_empty_units()
sorting_good = spikeinterface.curation.remove_excess_spikes(sorting_good,recording_chunk)

305


## Extracting the waveforms

In [5]:

job_kwargs = dict(n_jobs=4, chunk_duration="1s", progress_bar=True)
#waveforms = si.extract_waveforms(recording_chunk,sorting_KS3,folder="./waveformsblock1_7min",overwrite=True, ms_before=1., ms_after=2.,**job_kwargs)
waveforms = si.extract_waveforms(recording_chunk,sorting_good,folder='./sorting/waveforms_Folicacid10mg/',overwrite=True,**job_kwargs)
print(waveforms)

305


extract waveforms memmap:   0%|          | 0/180 [00:00<?, ?it/s]

WaveformExtractor: 1019 channels - 305 units - 1 segments
  before:30 after:40 n_per_units:500


In [4]:
waveforms = si.load_waveforms('./sorting/waveforms_Folicacid10mg/')
print(waveforms)
job_kwargs = dict(n_jobs=16, chunk_duration="1s", progress_bar=True)
sp.compute_spike_amplitudes(waveforms,load_if_exists=True,**job_kwargs)
metrics = qm.compute_quality_metrics(waveforms,load_if_exists=False,**job_kwargs)

WaveformExtractor: 1019 channels - 305 units - 1 segments
  before:30 after:40 n_per_units:500


/home/mmp/.local/lib/python3.10/site-packages/spikeinterface/qualitymetrics/misc_metrics.py:232: RuntimeWarning: divide by zero encountered in scalar divide
  snrs[unit_id] = np.abs(amplitude) / noise
/home/mmp/.local/lib/python3.10/site-packages/spikeinterface/qualitymetrics/misc_metrics.py:590: UserWarning: Units [0, 2, 8, 10, 12, 13, 14, 16, 22, 23, 28, 31, 33, 34, 37, 39, 45, 46, 53, 54, 55, 61, 63, 67, 68, 69, 76, 80, 81, 82, 84, 85, 86, 90, 91, 93, 96, 97, 98, 100, 103, 113, 114, 116, 119, 121, 122, 130, 134, 135, 136, 138, 140, 143, 144, 145, 147, 149, 150, 155, 156, 158, 160, 161, 162, 163, 165, 168, 183, 185, 187, 188, 191, 193, 197, 202, 204, 205, 207, 211, 212, 216, 219, 223, 224, 226, 229, 230, 236, 242, 246, 254, 255, 256, 259, 260, 261, 264, 267, 269, 272, 273, 276, 279, 282, 283, 285, 287, 289, 290, 295, 297, 299, 300, 303, 304, 305, 306, 310, 311, 312, 314, 317, 322, 325, 326, 331, 334, 335, 336, 340, 343, 348, 350, 353, 354, 359, 365, 366, 369, 370, 373, 375, 376, 381,

In [5]:
import mea_analysis_pipeline as msp
update_qual_metrics = msp.remove_violated_units(metrics)
non_violated_units  = update_qual_metrics.index.values

In [6]:
display(update_qual_metrics)

,num_spikes,firing_rate,presence_ratio,snr,isi_violations_ratio,isi_violations_count,rp_contamination,rp_violations,sliding_rp_violation,amplitude_cutoff,amplitude_median
7,806.0,4.477778,1.0,inf,0.000000,0.0,0.000000,0.0,0.185,0.082074,635.719299
9,676.0,3.755556,1.0,inf,0.000000,0.0,0.000000,0.0,0.185,0.004824,88.119507
15,644.0,3.577778,1.0,inf,0.000000,0.0,0.000000,0.0,0.055,0.024916,81.825256
17,935.0,5.194444,1.0,inf,0.000000,0.0,0.000000,0.0,0.045,0.003350,69.236755
32,839.0,4.661111,1.0,inf,0.000000,0.0,0.000000,0.0,0.085,0.005156,75.531006
...,...,...,...,...,...,...,...,...,...,...,...
546,513.0,2.850000,1.0,inf,0.000000,0.0,0.000000,0.0,0.080,0.000509,44.059753
548,1253.0,6.961111,1.0,inf,0.076433,2.0,0.122104,2.0,0.155,0.001038,50.354004
549,621.0,3.450000,1.0,inf,0.000000,0.0,0.000000,0.0,0.180,0.000417,37.765503
552,613.0,3.405556,1.0,inf,0.319345,2.0,0.795147,2.0,0.320,0.000422,44.059753


In [7]:
print(non_violated_units)

[  7   9  15  17  32  38  44  48  49  50  52  64  66  79 102 115 125 129
 139 151 170 172 177 182 190 194 198 201 210 215 225 227 232 243 244 253
 257 263 284 286 288 291 293 301 315 323 347 356 357 360 363 379 380 386
 388 395 399 402 405 419 425 428 431 441 442 453 470 484 497 499 506 520
 532 540 546 548 549 552 553]


In [8]:
len(non_violated_units)

79

In [9]:
waveform_good = waveforms.select_units(non_violated_units,new_folder='./sorting/waveformsgood_Folicacid10mg/')

In [10]:
metrics = qm.compute_quality_metrics(waveform_good,load_if_exists=False,**job_kwargs)

/home/mmp/.local/lib/python3.10/site-packages/spikeinterface/qualitymetrics/misc_metrics.py:232: RuntimeWarning: divide by zero encountered in scalar divide
  snrs[unit_id] = np.abs(amplitude) / noise


In [11]:
display(metrics)

,num_spikes,firing_rate,presence_ratio,snr,isi_violations_ratio,isi_violations_count,rp_contamination,rp_violations,sliding_rp_violation,amplitude_cutoff,amplitude_median
7,806.0,4.477778,1.0,inf,0.000000,0.0,0.000000,0.0,0.185,0.116709,635.719299
9,676.0,3.755556,1.0,inf,0.000000,0.0,0.000000,0.0,0.185,0.000383,88.119507
15,644.0,3.577778,1.0,inf,0.000000,0.0,0.000000,0.0,0.055,0.017724,81.825256
17,935.0,5.194444,1.0,inf,0.000000,0.0,0.000000,0.0,0.045,0.002369,62.942505
32,839.0,4.661111,1.0,inf,0.000000,0.0,0.000000,0.0,0.085,0.000525,75.531006
...,...,...,...,...,...,...,...,...,...,...,...
546,513.0,2.850000,1.0,inf,0.000000,0.0,0.000000,0.0,0.080,0.001561,44.059753
548,1253.0,6.961111,1.0,inf,0.076433,2.0,0.122104,2.0,0.155,0.000859,50.354004
549,621.0,3.450000,1.0,inf,0.000000,0.0,0.000000,0.0,0.180,0.000568,37.765503
552,613.0,3.405556,1.0,inf,0.319345,2.0,0.795147,2.0,0.320,0.000670,44.059753


In [12]:
waveform_good = None

In [13]:
waveform_good = si.load_waveforms('./sorting/waveformsgood_Folicacid10mg/')

In [14]:
metrics = qm.compute_quality_metrics(waveform_good,load_if_exists=False,**job_kwargs)

/home/mmp/.local/lib/python3.10/site-packages/spikeinterface/qualitymetrics/misc_metrics.py:232: RuntimeWarning: divide by zero encountered in scalar divide
  snrs[unit_id] = np.abs(amplitude) / noise


In [15]:
display(metrics)

,num_spikes,firing_rate,presence_ratio,snr,isi_violations_ratio,isi_violations_count,rp_contamination,rp_violations,sliding_rp_violation,amplitude_cutoff,amplitude_median
7,806.0,4.477778,1.0,inf,0.000000,0.0,0.000000,0.0,0.185,0.116709,635.719299
9,676.0,3.755556,1.0,inf,0.000000,0.0,0.000000,0.0,0.185,0.000383,88.119507
15,644.0,3.577778,1.0,inf,0.000000,0.0,0.000000,0.0,0.055,0.017724,81.825256
17,935.0,5.194444,1.0,inf,0.000000,0.0,0.000000,0.0,0.045,0.002369,62.942505
32,839.0,4.661111,1.0,inf,0.000000,0.0,0.000000,0.0,0.085,0.000525,75.531006
...,...,...,...,...,...,...,...,...,...,...,...
546,513.0,2.850000,1.0,inf,0.000000,0.0,0.000000,0.0,0.080,0.001561,44.059753
548,1253.0,6.961111,1.0,inf,0.076433,2.0,0.122104,2.0,0.155,0.000859,50.354004
549,621.0,3.450000,1.0,inf,0.000000,0.0,0.000000,0.0,0.180,0.000568,37.765503
552,613.0,3.405556,1.0,inf,0.319345,2.0,0.795147,2.0,0.320,0.000670,44.059753
